In a previous post, I showed a procedure for sampling from the posterior distribution in a Bayesian inference problem.
The goal was to infer the log-conductivity $q$ in the diffusion equation
$$-\nabla\cdot ke^q\nabla u = f$$
from a set of observations $u^o$ which have an error variance $\sigma$.
We had to first supply a prior density on the parameters $q$ which we took to have the form
$$-\ln\rho(q) = \alpha^2 R(q) + \text{constant}$$
where
$$R(q) = \frac{1}{2}\int_\Omega|Dq|^2dx.$$
Here I'd like to explore why this prior in particular may not be ideal and what happens if we instead use a prior functional based on the second derivative:
$$R(q) = \frac{1}{2}\int_\Omega(q^2 + \lambda^4|D^2u|^2)dx$$
The biharmonic operator is more challenging to discretize with finite elements.
We'll have to use an interior penalty DG approach, which builds off of Nitsche's method.

### Generating the exact data

We'll make the input data (sources $f$, external field $g$, exchange coefficient $h$, and true log-conductivity $q$) the same way as we did in the previous demo.

In [ ]:
import firedrake
mesh = firedrake.UnitSquareMesh(32, 32, diagonal='crossed')
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)

In [ ]:
import numpy as np
from numpy import random, pi as π
x = firedrake.SpatialCoordinate(mesh)

rng = random.default_rng(seed=1)
def random_fourier_series(std_dev, num_modes, exponent):
    from firedrake import sin, cos
    A = std_dev * rng.standard_normal((num_modes, num_modes))
    B = std_dev * rng.standard_normal((num_modes, num_modes))
    return sum([(A[k, l] * sin(π * (k * x[0] + l * x[1])) +
                 B[k, l] * cos(π * (k * x[0] + l * x[1])))
                / (1 + (k**2 + l**2)**(exponent/2))
                for k in range(num_modes)
                for l in range(int(np.sqrt(num_modes**2 - k**2)))])

In [ ]:
g = firedrake.Function(V).interpolate(random_fourier_series(1.0, 6, 1))

In [ ]:
import matplotlib.pyplot as plt
firedrake.trisurf(g);

We'll make the medium much more insulating (lower conductivity) near the center of the domain.

In [ ]:
from firedrake import inner, min_value, max_value, Constant
a = -Constant(8.)
r = Constant(1/4)
ξ = Constant((0.4, 0.5))
expr = a * max_value(0, 1 - inner(x - ξ, x - ξ) / r**2)
q_true = firedrake.Function(Q).interpolate(expr)

In [ ]:
firedrake.trisurf(q_true);

In order to make the effect most pronounced, we'll stick a blob of sources right next to this insulating patch.

In [ ]:
b = Constant(6.)
R = Constant(1/4)
η = Constant((0.7, 0.5))
expr = b * max_value(0, 1 - inner(x - η, x - η) / R**2)
f = firedrake.Function(V).interpolate(expr)

In [ ]:
firedrake.trisurf(f);

Compute the true field.

In [ ]:
from firedrake import exp, grad, dx, ds
k = Constant(1.)
h = Constant(10.)
u_true = firedrake.Function(V)
v = firedrake.TestFunction(V)
F = (
    (k * exp(q_true) * inner(grad(u_true), grad(v)) - f * v) * dx +
    h * (u_true - g) * v * ds
)

In [ ]:
opts = {
    'solver_parameters': {
        'ksp_type': 'preonly',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}
firedrake.solve(F == 0, u_true, **opts)

In [ ]:
firedrake.trisurf(u_true);

The true value of $u$ has a big hot spot in the insulating region, as we expect.

### Generating the observational data

For realistic problems, what we observe is the true solution plus some random noise $\xi$:

$$u_\text{obs} = u_\text{true} + \xi.$$

The ratio of the variance $\sigma$ of the noise to some scale of the solution, e.g. $\max_\Omega u_\text{true} - \min_\Omega u_\text{true}$, will determine the degree of accuracy that we can expect in the inferred field.
We have to remember again to synthesize the observations in the right way.
We want that
$$\mathbb{E}\left[\int_\Omega\xi(x)^2dx\right] = \sigma^2\cdot|\Omega|$$
which we can do using the Cholesky factorization.

In [ ]:
from firedrake import assemble
from firedrake.petsc import PETSc

area = assemble(Constant(1) * dx(mesh))

class NoiseGenerator:
    def __init__(
        self,
        function_space,
        covariance=None,
        generator=random.default_rng()
    ):
        if covariance is None:
            ϕ = firedrake.TrialFunction(function_space)
            ψ = firedrake.TestFunction(function_space)
            covariance = inner(ϕ, ψ) * dx

        M = assemble(covariance, mat_type='aij').M.handle
        ksp = PETSc.KSP().create()
        ksp.setOperators(M)
        ksp.setUp()

        pc = ksp.pc
        pc.setType(pc.Type.CHOLESKY)
        pc.setFactorSolverType(PETSc.Mat.SolverType.PETSC)
        pc.setFactorSetUpSolverType()
        L = pc.getFactorMatrix()
        pc.setUp()

        self.rng = generator
        self.function_space = function_space
        self.preconditioner = pc
        self.cholesky_factor = L

        self.rhs = firedrake.Function(self.function_space)
        self.noise = firedrake.Function(self.function_space)

    def __call__(self):
        z, ξ = self.rhs, self.noise
        N = len(z.dat.data_ro[:])
        z.dat.data[:] = self.rng.standard_normal(N)

        L = self.cholesky_factor
        with z.dat.vec_ro as Z:
            with ξ.dat.vec as Ξ:
                L.solveBackward(Z, Ξ)
                Ξ *= np.sqrt(area / N)

        return ξ.copy(deepcopy=True)

In [ ]:
white_noise_generator = NoiseGenerator(
    function_space=V,
    generator=random.default_rng(seed=1066)
)

We'll use a signal-to-noise ratio of 50 again.

In [ ]:
û = u_true.dat.data_ro[:]
signal = û.max() - û.min()
signal_to_noise = 50
σ = firedrake.Constant(signal / signal_to_noise)

u_obs = u_true.copy(deepcopy=True)
ξ = white_noise_generator()
u_obs += σ * ξ

The high-frequency noise you can see in the plot below is exactly what makes regularization necessary.

In [ ]:
firedrake.trisurf(u_obs);

### The biharmonic operator

Here we'd like to use the functional
$$R(q) = \int_\Omega\left(q^2 + \lambda^2|D^2q|^2\right)dx$$
which penalizes large values of the curvature $D^2q$.
Conventional finite element basis functions are continuous and piecewise-differentiable, but their derivatives have jump discontinuities across cell boundaries.
There are continuously-differentiable finite element bases which we could use to construct a conforming discretization of the curvature penalty.
I'll instead use a non-conforming discretization based on ordinary CG elements.
This approach is similar to how we used DG elements for the convection-diffusion equation.
For that problem, we applied Nitsche's method at all of the cell boundaries in order to make the solution continuous.
Here we'll instead apply Nitsche's method at all the cell boundaries to make the solution's gradient continuous.
I'm partly following [this paper](https://doi.org/10.1515/jnma-2023-0028) for discretization of the curvature penalty but working back to a minimization form.

In [ ]:
from firedrake import avg, jump, outer, dS

ϕ = firedrake.Function(Q)

λ = firedrake.Constant(1.0)
Dϕ = grad(ϕ)
DDϕ = grad(Dϕ)

h = firedrake.FacetArea(mesh)
vol = firedrake.CellVolume(mesh)

α = firedrake.Constant(4.0)
k = firedrake.Constant(Q.ufl_element().degree())
σ = 3 * α * k * (k - 1) / 8 * avg(h)**2 * avg(1 / vol)
σ_Γ = 3 * α * k * (k - 1) * h**2 / vol

ν = firedrake.FacetNormal(mesh)
J_cells = inner(DDϕ, DDϕ) * dx
J_facets = avg(inner(DDϕ, outer(ν, ν))) * jump(Dϕ, ν) * dS
J_facet_penalty = σ / avg(h) * jump(Dϕ, ν)**2 * dS
J_boundary = inner(Dϕ, ν) * inner(DDϕ, outer(ν, ν)) * ds
J_boundary_penalty = σ_Γ / h * inner(Dϕ, ν)**2 * ds

J_Δ = 0.5 * (J_cells - J_facets - J_boundary + J_facet_penalty + J_boundary_penalty)
J_2 = 0.5 * ϕ**2 * dx

J = J_2 + λ ** 4 * J_Δ

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
ξ = firedrake.Constant((0.5, 0.5))

y = x - ξ
f = y[0] * y[1]
L = f * ϕ * dx

E = J - L
F = firedrake.derivative(E, ϕ)

firedrake.solve(F == 0, ϕ)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(ϕ, axes=ax)
fig.colorbar(colors);

In [ ]:
from firedrake import derivative

M = derivative(derivative(J, ϕ), ϕ)
process = NoiseGenerator(
    function_space=Q,
    covariance=M,
    generator=np.random.default_rng(seed=1729),
)

In [ ]:
z = process()

In [ ]:
firedrake.trisurf(z);